# Machine Learning - Reseller Classification

## Project Scope:

The purpose of this project is to distinguish the resellers from the customers who purchased on the US website. Resellers harm the brand reputation, product market price, and inventory management of the company. Therefore, the company wants to identify and block the resellers' shipments.

The project involves a binary classification problem that may require feature engineering if necessary. The data source is a structured dataset in the database, which includes numerical and categorical features.

## Dataset column descriptions:

      'sales\_channel\_id': US sales channel id is 1, integer 

      'external\_customer\_id': customer id, integer 

      'email': customer email address, string

      'last\_shipping\_address\_address1': the shipping address 1 used in the last transaction, string

      'last\_shipping\_address\_address2': the shipping address 2 used in the last transaction, string

      'last\_shipping\_address\_city': the shipping city used in the last transaction, string

      'last\_shipping\_address\_zip': the shipping address zip code used in the last transaction, string

      'last\_shipping\_address\_country\_code': the shipping country code used in the last transaction, string

      'total\_orders': the total count of orders purchased by the customer, integer 

      'total\_units': the total count of item units purchased by the customer, integer 

      'total\_gross': the total gross sales spent by the customer, float

      'total\_discounts': the total discounts used by the customer, float

      'total\_returns': the total returns to the customer, float

      'total\_shipping': the total shipping spent by the customer, float

      'total\_taxes': the total taxes purchased by the customer, float

      'r\_score': recency score represents how recently a customer has made a purchase, score 1-5, integer 

      'f\_score': frequency score represents how often a customer makes a purchase, score 1-5, integer 

      'm\_score': monetary value score represents how much money a customer spends on purchases, score 1-5, integer

      'rfm\_score': r\_score + f\_score + m\_score, integer 

      'is\_reseller': 1 (reseller) or 0 (normal customer), this is the target, integer

## Preliminary Analysis:

The main features that are considered for the analysis are:

\- Total orders, units, gross sales, and discounts: These features reflect the reseller behavior of buying large quantities of products during the discount season.

\- Total returns: This feature indicates the reseller tendency of returning unsold products.

\- Total shipping and taxes: These features provide some information about the reseller location.

\- R\_score, F\_score, M\_score, RFM\_score: These features are derived from the recency, frequency, and monetary value of each customer's purchases and may help in training a model. The training process utilizes only R\_score, F\_score, and RFM\_score as the input features. This is based on the rationale that these three features encompass the information of M\_score. Including M\_score as an additional feature would result in a correlation problem.

  

One challenge in the reseller classification problem is to distinguish between loyal customers and resellers. Loyal customers spend a lot of money in total, but each transaction does not include many units. Resellers purchase multiple units in a single transaction. Therefore, two custom features are created to capture this difference:

\- Average units per order: This feature is obtained by dividing total units by total orders.

\- Average gross sales per order: This feature is obtained by dividing total gross sales by total orders.

The resellers may follow a different pattern in these two custom features compared to the loyal customers.

The preliminary training stage involves training a model with all the numerical features plus the two custom features. The model will be trained using PySpark MLlib and Keras Deep Learning, and the performance of different classifiers will be evaluated. This notebook focuses on PySpark MLlib models. The Keras Deep Learning models will be demonstrated in other notebooks going forward.

## Improvements:

Based on the feedback from the team who identified the resellers, I have analyzed the following criteria: email address, shipping address, and IP address. I have discovered that some resellers use multiple email accounts and vary their shipping addresses to avoid detection. However, these methods can be exposed by examining the email domain name and the embedding shipping address of the orders. The IP address is not a reliable indicator, as it can be easily changed by using a VPN. Therefore, I propose to create a model that considers both numerical and categorical features (excludes IP address) of the orders, and uses an embedding space to measure the distance between different shipping addresses. This will help us to detect the resellers' intentions more accurately.

## Define input and output

In [ ]:
model_name = "Reseller Classifier"
input_table_name = "customer"
output_table_name = "ml_resellers"

In [ ]:
# load table as a Spark DataFrame
customers = (spark.read
  .format("jdbc")
  .option("url", jdbcUrl)
  .option("dbtable", input_table_name)
  .option("user", user)
  .option("password", password)
  .load()
)

# take only US 2023 data
customers = customers.where('sales_channel_id = 1 AND CAST(last_transaction_date AS DATE) between "2023-01-01" and "2023-05-23" ')

In [ ]:
input_columns = ['sales_channel_id'
      ,'external_customer_id'
      ,'email'
      ,'last_shipping_address_address1'
      ,'last_shipping_address_address2'
      ,'last_shipping_address_city'
      ,'last_shipping_address_zip'
      ,'last_shipping_address_country_code'
      ,'total_orders'
      ,'total_units'
      ,'total_gross'
      ,'total_discounts'
      ,'total_returns'
      ,'total_shipping'
      ,'total_taxes'
      ,'r_score'
      ,'f_score'
      ,'rfm_score'
      ,'is_reseller'
      ]

## Create Custom Features

In [ ]:
import pyspark.sql.functions as F

df = customers.select(
    [customers[col] for col in input_columns]
    + [
        F.substring_index(
            F.substring_index(F.lower(customers["email"]), "@", -1), ".", 1
        ).alias("email_domain")
    ]
    + [
        F.concat(
            F.coalesce(F.lower(customers["last_shipping_address_address1"]), F.lit("")),
            F.lit(" "),
            F.coalesce(F.lower(customers["last_shipping_address_address2"]), F.lit("")),
            F.lit(" "),
            F.coalesce(F.lower(customers["last_shipping_address_city"]), F.lit("")),
            F.lit(" "),
            F.coalesce(F.lower(customers["last_shipping_address_country_code"]), F.lit("")),
            F.lit(" "),
            F.coalesce(F.lower(customers["last_shipping_address_zip"]), F.lit("")),
        ).alias("address")
    ]
).withColumn("email_domain_address", F.array("email_domain", "address"))

## Data Cleansing

In [ ]:
IDENTIFIERS = ['external_customer_id', 'email']
CONTINUOUS_COLUMNS = [
  'total_orders',
  'total_units',
  'total_gross',
  'total_discounts',
  'total_returns',
  'total_shipping',
  'total_taxes',
  'r_score',
  'f_score',
  'rfm_score'
]
CATEGORICAL_COLUMN = 'email_domain_address'
TARGET_COLUMN = ['is_reseller']

In [ ]:
customers = df.dropna(
  how='any',
  subset=[x for x in IDENTIFIERS + CONTINUOUS_COLUMNS + TARGET_COLUMN] + [CATEGORICAL_COLUMN]
)

In [ ]:
customers = customers.withColumn(
  'units_per_order', F.col('total_units') * 1.0 / F.col('total_orders')
).withColumn(
  'gross_per_order', F.col('total_gross') * 1.0 / F.col('total_orders')
)

customers = customers.fillna(0.0, subset=['units_per_order', 'gross_per_order'])

CONTINUOUS_COLUMNS += ['units_per_order', 'gross_per_order']

In [ ]:
CONTINUOUS_COLUMNS

['total_orders',
 'total_units',
 'total_gross',
 'total_discounts',
 'total_returns',
 'total_shipping',
 'total_taxes',
 'r_score',
 'f_score',
 'rfm_score',
 'units_per_order',
 'gross_per_order']

## Train Test Split

In [ ]:
resellers = customers.where('sales_channel_id = 1 and is_reseller = 1')
normal_customers = customers.where('sales_channel_id = 1 and is_reseller = 0')

train_resellers, test_resellers, val_resellers = resellers.randomSplit([0.8, 0.1, 0.1], seed=42)
train_normal, test_normal, val_normal = normal_customers.randomSplit([0.8, 0.1, 0.1], seed=42)

train = train_resellers.union(train_normal).orderBy(F.rand(seed=42))
test = test_resellers.union(test_normal).orderBy(F.rand(seed=42))
val = val_resellers.union(val_normal).orderBy(F.rand(seed=42))

train.cache()

DataFrame[sales_channel_id: int, external_customer_id: bigint, customer_type: string, email: string, first_name: string, last_name: string, last_shipping_address_address1: string, last_shipping_address_address2: string, last_shipping_address_city: string, last_shipping_address_country: string, last_shipping_address_phone: string, last_shipping_address_province: string, last_shipping_address_zip: string, last_shipping_address_country_code: string, last_shipping_address_province_code: string, first_transaction_date: timestamp, first_transaction_id: bigint, last_transaction_date: timestamp, last_transaction_id: bigint, last_transaction_ip: string, total_orders: int, total_units: int, total_gross: decimal(19,4), total_discounts: decimal(19,4), total_returns: decimal(19,4), total_shipping: decimal(19,4), total_taxes: decimal(19,4), r_score: int, f_score: int, m_score: int, rfm_score: int, is_reseller: int, email_domain: string, address: string, email_domain_address: array<string>, units_per

## Create a Pipeline

In [ ]:
imput_columns = [(x + '_i') for x in CONTINUOUS_COLUMNS if x not in ['units_per_order', 'gross_per_order']]

In [ ]:
from pyspark.ml import Pipeline
import pyspark.ml.feature as MF

embedding_size = 100

imputer = MF.Imputer(
  strategy='mean',
  inputCols=[x for x in CONTINUOUS_COLUMNS if x not in ['units_per_order', 'gross_per_order']],
  outputCols=imput_columns
)

embedding = MF.Word2Vec(
    vectorSize=embedding_size,
    inputCol='email_domain_address',
    outputCol='embedded'
)

continuous_assembler = MF.VectorAssembler(
  inputCols=imput_columns + ['units_per_order', 'gross_per_order', 'embedded'],
  outputCol='continuous'
)

continuous_scaler = MF.StandardScaler(
  inputCol='continuous',
  outputCol='features'
)

customers_pipeline = Pipeline(
  stages=[imputer, embedding, continuous_assembler, continuous_scaler]
)

customers_pipeline_model = customers_pipeline.fit(train)
customers_features = customers_pipeline_model.transform(train)

##LogisticRegression

In [ ]:
from pyspark.ml.classification import LogisticRegression
import mlflow

clf = LogisticRegression(
  featuresCol='features',
  labelCol='is_reseller',
  predictionCol='prediction'
)

customers_pipeline.setStages(
  [
    imputer, 
    embedding,
    continuous_assembler, 
    continuous_scaler,
    clf
  ]
)

# Start an MLflow run
mlflow.start_run()

# Train a model
customers_pipeline_model = customers_pipeline.fit(train)

# Predictions
results = customers_pipeline_model.transform(val)

# Log metrics
model = customers_pipeline_model.stages[-1]
metrics = model.evaluate(results.select('email', 'is_reseller', 'features'))

mlflow.log_metric("val_accuracy", metrics.accuracy)
mlflow.log_metric("val_precision", metrics.precisionByLabel[1])
mlflow.log_metric("val_recall", metrics.recallByLabel[1])

# Log the model
mlflow.spark.log_model(customers_pipeline_model, "LogisticRegression_model")

# Close the MLflow run
mlflow.end_run()

2023/05/25 02:56:25 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


##RandomForest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

clf = RandomForestClassifier(
  featuresCol='features',
  labelCol='is_reseller',
  predictionCol='prediction'
)

customers_pipeline.setStages(
  [
    imputer, 
    embedding,
    continuous_assembler, 
    continuous_scaler,
    clf
  ]
)

# Start an MLflow run
mlflow.start_run()

# Train a model
customers_pipeline_model = customers_pipeline.fit(train)

# Predictions
results = customers_pipeline_model.transform(val)

# Log metrics
model = customers_pipeline_model.stages[-1]
metrics = model.evaluate(results.select('email', 'is_reseller', 'features'))

mlflow.log_metric("val_accuracy", metrics.accuracy)
mlflow.log_metric("val_precision", metrics.precisionByLabel[1])
mlflow.log_metric("val_recall", metrics.recallByLabel[1])

# Log the model
mlflow.spark.log_model(customers_pipeline_model, "RandomForest_model")

# Close the MLflow run
mlflow.end_run()

2023/05/25 04:57:59 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().


##GBTClassifier

In [ ]:
import mlflow

from pyspark.ml.classification import GBTClassifier
from pyspark.mllib.evaluation import MulticlassMetrics

clf = GBTClassifier(
  featuresCol='features',
  labelCol='is_reseller',
  predictionCol='prediction'
)

customers_pipeline.setStages(
  [
    imputer, 
    embedding,
    continuous_assembler, 
    continuous_scaler,
    clf
  ]
)

# Start an MLflow run
mlflow.start_run()

# Train a model
customers_pipeline_model = customers_pipeline.fit(train)

# Predictions
results = customers_pipeline_model.transform(val)

# Extract the predicted labels and true labels from the predictions DataFrame
predictionAndLabels = results.select('prediction', F.col('is_reseller').cast("double")).rdd

# Create MulticlassMetrics object
metrics = MulticlassMetrics(predictionAndLabels)

# Log metrics
mlflow.log_metric("val_accuracy", metrics.accuracy)
mlflow.log_metric("val_precision", metrics.precision(1))
mlflow.log_metric("val_recall", metrics.recall(1))

# Log the model
mlflow.spark.log_model(customers_pipeline_model, "GBT_model")

# Close the MLflow run
mlflow.end_run()

/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
2023/05/25 19:13:03 INFO mlflow.spark: Inferring pip requirements by reloading the logged model from the databricks artifact repository, which can be time-consuming. To speed up, explicitly specify the conda_env or pip_requirements when calling log_model().
/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


##Load Best Model

### Validation dataset Evaluation

In [ ]:
import mlflow
logged_model = 'runs:/GBT_model'

# Load model
loaded_model = mlflow.spark.load_model(logged_model)

# Perform inference via model.transform()
results = loaded_model.transform(val)

# Extract the predicted labels and true labels from the predictions DataFrame
predictionAndLabels = results.select('prediction', F.col('is_reseller').cast("double")).rdd

# Create MulticlassMetrics object
metrics = MulticlassMetrics(predictionAndLabels)

# validation metrics
print(metrics.accuracy, metrics.precision(1), metrics.recall(1))

2023/05/25 03:16:53 INFO mlflow.spark: 'runs:/a2b45aae0d854e5b81cb34caebd12e07/GBT_model' resolved as 'dbfs:/databricks/mlflow-tracking/4401469133110519/a2b45aae0d854e5b81cb34caebd12e07/artifacts/GBT_model'
/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


0.9992584987451517 0.9090909090909091 0.75


### Test dataset Evaluation

In [ ]:
import mlflow
logged_model = 'runs:/GBT_model'

# Load model
loaded_model = mlflow.spark.load_model(logged_model)

# Perform inference via model.transform()
results = loaded_model.transform(test)

# Extract the predicted labels and true labels from the predictions DataFrame
predictionAndLabels = results.select('prediction', F.col('is_reseller').cast("double")).rdd

# Create MulticlassMetrics object
metrics = MulticlassMetrics(predictionAndLabels)

# validation metrics
print(metrics.accuracy, metrics.precision(1), metrics.recall(1))

2023/05/25 03:17:13 INFO mlflow.spark: 'runs:/a2b45aae0d854e5b81cb34caebd12e07/GBT_model' resolved as 'dbfs:/databricks/mlflow-tracking/4401469133110519/a2b45aae0d854e5b81cb34caebd12e07/artifacts/GBT_model'
/databricks/spark/python/pyspark/sql/context.py:165: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


0.9994229326562409 0.9333333333333333 0.7777777777777778
